# Setup Comprehend Through AWS Console 

<img src="img/comprehend.png" width="80%" align="left">

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(
   retries = {
      'max_attempts': 10,
      'mode': 'adaptive'
   }
)

iam = boto3.client('iam', config=config)
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
comprehend = boto3.client('comprehend')

In [3]:
%store -r noheader_train_s3_uri

print(noheader_train_s3_uri)

s3://sagemaker-us-west-2-140773038493/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv


In [4]:
!aws s3 ls $noheader_train_s3_uri

2020-07-25 16:10:28   13639640 amazon_reviews_us_Digital_Software_v1_00_noheader.csv


# Create Data Access Role for Comprehend

## Create Policy

In [5]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "comprehend.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

## Create Role and Attach Policies

In [6]:
iam_comprehend_role_name = 'DSOAWS_Comprehend'

In [7]:
import json
import time

from botocore.exceptions import ClientError

try:
    iam_role_comprehend = iam.create_role(
        RoleName=iam_comprehend_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Comprehend Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_comprehend = iam.get_role(RoleName=iam_comprehend_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)
        
time.sleep(30)

In [8]:
comprehend_s3_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        }
    ]
}

print(comprehend_s3_policy_doc)

{'Version': '2012-10-17', 'Statement': [{'Action': ['s3:GetObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-140773038493/*'], 'Effect': 'Allow'}, {'Action': ['s3:ListBucket'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-140773038493'], 'Effect': 'Allow'}, {'Action': ['s3:PutObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-west-2-140773038493/*'], 'Effect': 'Allow'}]}


# Attach Policy to Role

In [9]:
import time

response = iam.put_role_policy(
    RoleName=iam_comprehend_role_name,
    PolicyName='DSOAWS_ComprehendPolicyToS3',
    PolicyDocument=json.dumps(comprehend_s3_policy_doc)
)

print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': '8c274f6d-6c51-482b-887c-09dbfd2a1341', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8c274f6d-6c51-482b-887c-09dbfd2a1341', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Sat, 25 Jul 2020 16:30:59 GMT'}, 'RetryAttempts': 0}}


# Train the Model

In [10]:
prefix = 'models'

s3_output_job = 's3://{}/{}/{}'.format(bucket, prefix, 'comprehend/output')
print(s3_output_job)

s3://sagemaker-us-west-2-140773038493/models/comprehend/output


In [11]:
iam_role_comprehend_arn = iam_role_comprehend['Role']['Arn']

In [12]:
import datetime
import time

id = str(datetime.datetime.now().strftime("%s"))

training_job = comprehend.create_document_classifier(
    DocumentClassifierName='Amazon-Customer-Reviews-Classifier-'+ id,
    DataAccessRoleArn=iam_role_comprehend_arn,
    InputDataConfig={
        'S3Uri': noheader_train_s3_uri
    },
    OutputDataConfig={
        'S3Uri': s3_output_job
    },
    LanguageCode='en'
)

time.sleep(30)

In [13]:
import time

jobArn = training_job['DocumentClassifierArn']

max_time = time.time() + 3 * 60 * 60 # 3 hours
while time.time() < max_time:
    describe_custom_classifier = comprehend.describe_document_classifier(
        DocumentClassifierArn = jobArn
    )
    status = describe_custom_classifier["DocumentClassifierProperties"]["Status"]
    print("Custom classifier: {}".format(status))
    
    if status == "TRAINED" or status == "IN_ERROR":
        print('')
        print('Status {}'.format(status))
        print('')
        print(describe_custom_classifier["DocumentClassifierProperties"])
        break
        
    time.sleep(10)

Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Cu

# _Please Wait Until the ^^ Classifier ^^ is Trained Above._

# Show Results of the Classifier

In [14]:
print(describe_custom_classifier["DocumentClassifierProperties"])

{'DocumentClassifierArn': 'arn:aws:comprehend:us-west-2:140773038493:document-classifier/Amazon-Customer-Reviews-Classifier-1595694690', 'LanguageCode': 'en', 'Status': 'TRAINED', 'SubmitTime': datetime.datetime(2020, 7, 25, 16, 31, 30, 653000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 7, 25, 17, 19, 8, 92000, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2020, 7, 25, 16, 37, 45, 940000, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2020, 7, 25, 17, 17, 52, 828000, tzinfo=tzlocal()), 'InputDataConfig': {'S3Uri': 's3://sagemaker-us-west-2-140773038493/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv'}, 'OutputDataConfig': {'S3Uri': 's3://sagemaker-us-west-2-140773038493/models/comprehend/output/140773038493-CLR-1d24f83da673bc8be2ede7cd1ec487e3/output/output.tar.gz'}, 'ClassifierMetadata': {'NumberOfLabels': 5, 'NumberOfTrainedDocuments': 27905, 'NumberOfTestDocuments': 3100, 'EvaluationMetrics': {'Accuracy': 0.5306, 'Precision': 0.5267, 'Recal

In [15]:
model_arn = describe_custom_classifier["DocumentClassifierProperties"]["DocumentClassifierArn"]
print(model_arn)

arn:aws:comprehend:us-west-2:140773038493:document-classifier/Amazon-Customer-Reviews-Classifier-1595694690


In [16]:
import os
#Retrieve the S3URI from the model output and create jobkey variable.
job_output = describe_custom_classifier["DocumentClassifierProperties"]["OutputDataConfig"]["S3Uri"]
print(job_output)

path_prefix = 's3://{}/'.format(bucket)

job_key = os.path.relpath(job_output, path_prefix)

print(job_key)

s3://sagemaker-us-west-2-140773038493/models/comprehend/output/140773038493-CLR-1d24f83da673bc8be2ede7cd1ec487e3/output/output.tar.gz
models/comprehend/output/140773038493-CLR-1d24f83da673bc8be2ede7cd1ec487e3/output/output.tar.gz


In [17]:
#Download the model metrics
s3 = boto3.resource('s3')

s3.Bucket(bucket).download_file(job_key, './output.tar.gz')

In [18]:
#Unpack the gzip file
!tar xvzf ./output.tar.gz

output/
output/confusion_matrix.json


In [19]:
import json

with open('./output/confusion_matrix.json') as json_file:
    data = json.load(json_file)
print(json.dumps(data, indent=2, default=str))

{
  "confusion_matrix": [
    [
      389,
      139,
      70,
      14,
      8
    ],
    [
      165,
      263,
      146,
      34,
      12
    ],
    [
      72,
      124,
      269,
      124,
      31
    ],
    [
      20,
      33,
      126,
      278,
      163
    ],
    [
      10,
      11,
      22,
      131,
      446
    ]
  ],
  "labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ],
  "type": "multi_class",
  "all_labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ]
}


In [20]:
!pip install tabulate

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [21]:
from IPython.display import HTML, display
import tabulate
table = [['', '1', '2', '3', '4', '5', '(Predicted)'],
         ['1', data['confusion_matrix'][0][0], data['confusion_matrix'][0][1], data['confusion_matrix'][0][2], data['confusion_matrix'][0][3], data['confusion_matrix'][0][4]],
         ['2', data['confusion_matrix'][1][0], data['confusion_matrix'][1][1], data['confusion_matrix'][1][2], data['confusion_matrix'][1][3], data['confusion_matrix'][1][4]],
         ['3', data['confusion_matrix'][2][0], data['confusion_matrix'][2][1], data['confusion_matrix'][2][2], data['confusion_matrix'][2][3], data['confusion_matrix'][2][4]],
         ['4', data['confusion_matrix'][3][0], data['confusion_matrix'][3][1], data['confusion_matrix'][3][2], data['confusion_matrix'][3][3], data['confusion_matrix'][3][4]],
         ['5', data['confusion_matrix'][4][0], data['confusion_matrix'][4][1], data['confusion_matrix'][4][2], data['confusion_matrix'][4][3], data['confusion_matrix'][4][4]],
         ['(Actual)']]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

,1,2,3,4,5,(Predicted)
1,389,139,70,14,8,
2,165,263,146,34,12,
3,72,124,269,124,31,
4,20,33,126,278,163,
5,10,11,22,131,446,
(Actual),,,,,,


# Deploy Endpoint

In [22]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

comprehend_endpoint_name = 'comprehend-inference-ep-' + timestamp_suffix

inference_endpoint_response = comprehend.create_endpoint(
    EndpointName=comprehend_endpoint_name,
    ModelArn = model_arn,
    DesiredInferenceUnits = 1
)

In [23]:
comprehend_endpoint_arn = inference_endpoint_response["EndpointArn"]
print(comprehend_endpoint_arn)

arn:aws:comprehend:us-west-2:140773038493:document-classifier-endpoint/comprehend-inference-ep-25-17-19-22


# Pass Variables to the Next Notebook(s)

In [24]:
%store comprehend_endpoint_arn

Stored 'comprehend_endpoint_arn' (str)


# Predict with Endpoint

In [25]:
describe_response = comprehend.describe_endpoint(
    EndpointArn = comprehend_endpoint_arn
)
print(describe_response)

{'EndpointProperties': {'EndpointArn': 'arn:aws:comprehend:us-west-2:140773038493:document-classifier-endpoint/comprehend-inference-ep-25-17-19-22', 'Status': 'CREATING', 'ModelArn': 'arn:aws:comprehend:us-west-2:140773038493:document-classifier/Amazon-Customer-Reviews-Classifier-1595694690', 'DesiredInferenceUnits': 1, 'CurrentInferenceUnits': 0, 'CreationTime': datetime.datetime(2020, 7, 25, 17, 19, 22, 417000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 7, 25, 17, 19, 22, 417000, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': '040a22fa-d78f-4302-888b-a5f8a7c0af61', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '040a22fa-d78f-4302-888b-a5f8a7c0af61', 'content-type': 'application/x-amz-json-1.1', 'content-length': '408', 'date': 'Sat, 25 Jul 2020 17:19:22 GMT'}, 'RetryAttempts': 0}}


In [26]:
import time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_response = comprehend.describe_endpoint(
        EndpointArn = comprehend_endpoint_arn
    )
    status = describe_response["EndpointProperties"]["Status"]
    print("Endpoint: {}".format(status))
    
    if status == "IN_SERVICE" or status == "IN_ERROR":
        break
        
    time.sleep(5)

Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CR

In [27]:
txt = """I loved it!  I will recommend this to everyone."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "5",
      "Score": 0.8745999932289124
    },
    {
      "Name": "4",
      "Score": 0.0892999991774559
    },
    {
      "Name": "1",
      "Score": 0.013700000010430813
    }
  ],
  "ResponseMetadata": {
    "RequestId": "7518dc3c-b291-4401-a342-0edd7fdaa14b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "7518dc3c-b291-4401-a342-0edd7fdaa14b",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Sat, 25 Jul 2020 17:28:24 GMT"
    },
    "RetryAttempts": 0
  }
}


In [28]:
txt = """It's OK."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "3",
      "Score": 0.7190999984741211
    },
    {
      "Name": "4",
      "Score": 0.11829999834299088
    },
    {
      "Name": "2",
      "Score": 0.09719999879598618
    }
  ],
  "ResponseMetadata": {
    "RequestId": "cfa68acb-dea3-4882-8aa1-9593f631c182",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "cfa68acb-dea3-4882-8aa1-9593f631c182",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Sat, 25 Jul 2020 17:28:24 GMT"
    },
    "RetryAttempts": 0
  }
}


In [29]:
txt = """Really bad.  I hope they don't make this anymore."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "1",
      "Score": 0.44859999418258667
    },
    {
      "Name": "2",
      "Score": 0.26989999413490295
    },
    {
      "Name": "3",
      "Score": 0.22769999504089355
    }
  ],
  "ResponseMetadata": {
    "RequestId": "b0ed754a-62c3-4ae3-8b85-8b438726c3be",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "b0ed754a-62c3-4ae3-8b85-8b438726c3be",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "136",
      "date": "Sat, 25 Jul 2020 17:28:24 GMT"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

# Navigate to Comprehend in the AWS Console
![Comprehend Console](img/comprehend-console.png)